## Importing Libraries


In [5]:
from __future__ import print_function
import cv2 as cv
import numpy as np
import glob
import os
import pickle

## Reading Database folder using glob and calculating kp and dc 



In [2]:
print("Please choose detector form SIFT AND ORB")
detector_name = input()
print("Choosed Detector:",detector_name)

Please choose detector form SIFT AND ORB
Choosed Detector: ORB


In [3]:
if detector_name == "SIFT":
    detector = cv.SIFT_create()
elif detector_name == "ORB":
    detector = cv.ORB_create()

In [7]:
images_data = []
for img in glob.glob("C:/Users/piyus/OneDrive/Documents/Office/24Sep/storing_keypoints_descriptors_yml/database/*.jpg"):
        n = cv.imread(img, cv.IMREAD_GRAYSCALE)
        kp, dc = detector.detectAndCompute(n, None)
        keypoints = []
        for p in kp:
            temp = (p.pt, p.size, p.angle, p.response, p.octave, p.class_id)
            keypoints.append(temp)

        img_info={
            "image_name" : os.path.basename(img),
            "image_kp"   : keypoints,
            "image_dp"   : dc
        }
        images_data.append(img_info)

-------------------------------------------------------  # DATABASE LOADED # ----------------------------------------------------------------------

### Saving Database in pickle


In [8]:
with open("sources_data.pickle", mode="wb") as f:
    pickle.dump(images_data, f)

# Using stored database for finding 

### Loading query image

In [9]:
#Load Query image
Query_img = cv.imread("Query.jpg ")
Query_keypoint, Query_descriptor = detector.detectAndCompute(Query_img, None)

### Getting data from stored file and converting keypoints to objects


In [30]:
def get_sources():
    """Get source's features from file
        Returns:
            sources(list): source's keypoints, descriptors,and img
        """
    #Get feature point information from a file

    with open("sources_data.pickle", mode="rb") as f:
        images_data_loaded = pickle.load(f)
    for items in images_data_loaded:
        
        #Restore keypoints to original structure

        keypoints = []
        for p in items["image_kp"]:
            temp = cv.KeyPoint(
                x=p[0][0],
                y=p[0][1],
                size=p[1],
                angle=p[2],
                response=p[3],
                octave=p[4],
                class_id=p[5],
            )
            keypoints.append(temp)
        items["image_kp"] = keypoints

    return images_data_loaded

In [35]:
images_data_loaded = get_sources()
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)

## Feature Matching


In [34]:
for n in images_data_loaded:
    knn_matches = matcher.knnMatch(n["image_dp"], Query_descriptor, k=2)
    #Thin out data
    ratio = 0.5
    matched_keypoints = []
    for m, n in knn_matches:
        if m.distance < ratio * n.distance:
            matched_keypoints.append([m])

    #Output the result image when there are more good than any threshold
    if len(matched_keypoints) > 20:
        print("img found")
            

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\flann\src\miniflann.cpp:336: error: (-210:Unsupported format or combination of formats) in function 'cv::flann::buildIndex_'
> type=0
> 